In [1]:
# ---------------------------------------------------------
# SpaceX Launch Records Dashboard (Plotly Dash)
# Tasks covered:
# 1) Launch Site dropdown
# 2) Pie chart callback (success counts)
# 3) Payload range slider
# 4) Scatter plot callback (success vs payload)
# ---------------------------------------------------------

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


In [2]:
# ---------------------------------------------------------
# REQUIRED: spacex_df must exist before running the app.
# It should include at least these columns:
#   'Launch Site', 'class', 'Payload Mass (kg)'
# where class: 1=Success, 0=Failure
# ---------------------------------------------------------
# Example (you already have yours from the lab):
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# ----------------------------
# Helper values for slider
# ----------------------------
min_payload = int(spacex_df["Payload Mass (kg)"].min())
max_payload = int(spacex_df["Payload Mass (kg)"].max())

# ----------------------------
# Build dropdown options
# ----------------------------
launch_sites = sorted(spacex_df["Launch Site"].unique())
site_options = [{"label": "All Sites", "value": "ALL"}] + [
    {"label": site, "value": site} for site in launch_sites
]


In [3]:
# ----------------------------
# Dash app
# ----------------------------
app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        html.H1(
            "SpaceX Launch Records Dashboard",
            style={"textAlign": "center", "color": "#503D36", "fontSize": 40},
        ),

        # TASK 1: Launch Site dropdown
        dcc.Dropdown(
            id="site-dropdown",
            options=site_options,
            value="ALL",
            placeholder="Select a Launch Site here",
            searchable=True,
            style={"width": "80%", "padding": "3px", "fontSize": "20px"},
        ),
        html.Br(),

        # TASK 2: Pie chart output
        html.Div(dcc.Graph(id="success-pie-chart")),
        html.Br(),

        html.P("Payload range (kg):", style={"fontSize": "18px"}),

        # TASK 3: Payload range slider
        dcc.RangeSlider(
            id="payload-slider",
            min=min_payload,
            max=max_payload,
            step=1000,
            marks={
                min_payload: str(min_payload),
                max_payload: str(max_payload),
            },
            value=[min_payload, max_payload],
        ),
        html.Br(),

        # TASK 4: Scatter plot output
        html.Div(dcc.Graph(id="success-payload-scatter-chart")),
    ]
)


In [4]:
# ---------------------------------------------------------
# TASK 2: Callback for Pie Chart
#  - If ALL: show total successes by site (counts of class==1 grouped by site)
#  - Else: show success vs failure for selected site
# ---------------------------------------------------------
@app.callback(
    Output("success-pie-chart", "figure"),
    Input("site-dropdown", "value"),
)
def update_pie_chart(selected_site):
    if selected_site == "ALL":
        # Count ONLY successes per launch site
        success_counts = (
            spacex_df[spacex_df["class"] == 1]
            .groupby("Launch Site", as_index=False)["class"]
            .count()
            .rename(columns={"class": "Success Count"})
        )

        fig = px.pie(
            success_counts,
            values="Success Count",
            names="Launch Site",
            title="Total Successful Launches by Site",
        )
        return fig

    # Selected site: show success vs failure
    site_df = spacex_df[spacex_df["Launch Site"] == selected_site]
    outcome_counts = (
        site_df["class"]
        .value_counts()
        .reset_index()
        .rename(columns={"index": "Outcome", "class": "Count"})
    )
    # Map 0/1 to labels
    outcome_counts["Outcome"] = outcome_counts["Outcome"].map({1: "Success", 0: "Failure"})

    fig = px.pie(
        outcome_counts,
        values="Count",
        names="Outcome",
        title=f"Success vs Failure for {selected_site}",
    )
    return fig


In [5]:
# ---------------------------------------------------------
# TASK 4: Callback for Scatter Plot
#  - Filter by site (ALL or one site)
#  - Filter by payload range
#  - Scatter: x=Payload, y=class, color=class (Success/Failure)
# ---------------------------------------------------------
@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [
        Input("site-dropdown", "value"),
        Input("payload-slider", "value"),
    ],
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range

    # Filter by payload range first
    filtered_df = spacex_df[
        (spacex_df["Payload Mass (kg)"] >= low) & (spacex_df["Payload Mass (kg)"] <= high)
    ]

    # Then filter by site (if not ALL)
    if selected_site != "ALL":
        filtered_df = filtered_df[filtered_df["Launch Site"] == selected_site]

    # Make class labels nicer for legend
    plot_df = filtered_df.copy()
    plot_df["Outcome"] = plot_df["class"].map({1: "Success", 0: "Failure"})

    fig = px.scatter(
        plot_df,
        x="Payload Mass (kg)",
        y="class",
        color="Outcome",
        hover_data=["Launch Site"],
        title=(
            "Payload vs Outcome for All Sites"
            if selected_site == "ALL"
            else f"Payload vs Outcome for {selected_site}"
        ),
    )

    # Optional: make y-axis show 0/1 clearly
    fig.update_yaxes(tickmode="array", tickvals=[0, 1], ticktext=["Failure (0)", "Success (1)"])
    return fig


In [7]:
# ---------------------------------------------------------
# Run
# ---------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True)
